# BIGSI Test 1

This is a test of the software `bigsi` <https://bigsi.readme.io/>, specifically by following the tutorial <https://bigsi.readme.io/docs/your-first-bigsi>.

# Step 1

Step 1 involves downloading the fastq files and counting kmers. This makes use of the software `fasterq-dump` to download fastq files and `jellyfish` to count kmers. The genomes to download are stored in the file `genomes.txt`.

In [1]:
NOTEBOOK_DIR=`git rev-parse --show-toplevel`

In [2]:
cd $NOTEBOOK_DIR
rm -rf kmers
mkdir kmers
pushd kmers

~/workspace/bigsi-examples/kmers ~/workspace/bigsi-examples


In [4]:
kmer_size=31
threads=16

for accession in `cat ../genomes.txt`;
do
    echo "Working on $accession"
    
    fastq_file=${accession}.fastq
    cortex_out=${accession}.ctx
    cortex_log=${accession}_mccortex.log
    
    set -x
    fasterq-dump -s -o ${fastq_file} "$accession"
    mccortex31 build -t ${threads} -k ${kmer_size} -s ${accession} --seqi ${fastq_file} ${cortex_out} 2> ${cortex_log}.err 1> ${cortex_log}
    set +x
done

echo "Done"

Working on SRR9842706
+ fasterq-dump -s -o SRR9842706.fastq SRR9842706
2019-10-01T22:35:49 fasterq-dump.2.10.0 err: fasterq-dump.c fastdump_csra() checking ouput-file 'SRR9842706.fastq' -> RC(rcExe,rcFile,rcPacking,rcName,rcExists)
+ mccortex31 build -t 16 -k 31 -s SRR9842706 --seqi SRR9842706.fastq SRR9842706.ctx
+ set +x
Working on SRR10140545
+ fasterq-dump -s -o SRR10140545.fastq SRR10140545
2019-10-01T22:35:52 fasterq-dump.2.10.0 err: fasterq-dump.c fastdump_csra() checking ouput-file 'SRR10140545.fastq' -> RC(rcExe,rcFile,rcPacking,rcName,rcExists)
+ mccortex31 build -t 16 -k 31 -s SRR10140545 --seqi SRR10140545.fastq SRR10140545.ctx
+ set +x
Working on SRR10140517
+ fasterq-dump -s -o SRR10140517.fastq SRR10140517
2019-10-01T22:35:54 fasterq-dump.2.10.0 err: fasterq-dump.c fastdump_csra() checking ouput-file 'SRR10140517.fastq' -> RC(rcExe,rcFile,rcPacking,rcName,rcExists)
+ mccortex31 build -t 16 -k 31 -s SRR10140517 --seqi SRR10140517.fastq SRR10140517.ctx
+ set +x
Working on 

Awesome. We've now got files named like `SRR9842706.ctx` in our directory containing the cortex graph/kmers.

# Step 2

In step 2, we'll look at building the `bigsi` indexes for each of these kmer counts.

In [5]:
number_hashes=3
m_value=25000000

mkdir bigsi
pushd bigsi

~/workspace/bigsi-examples/kmers/bigsi ~/workspace/bigsi-examples/kmers ~/workspace/bigsi-examples


Let's setup the bigsi index configuration.

In [6]:
cat > berkeleydb.yaml << EOF
## Example config using berkeleyDB
h: ${number_hashes}
k: ${kmer_size}
m: ${m_value}
storage-engine: berkeleydb
storage-config:
  filename: bigsi.db
  flag: "c" ## Change to 'r' for read-only access
EOF

export BIGSI_CONFIG=berkeleydb.yaml

Now, let's construct the blooom filters.

In [ ]:
# Link all .ctx files into current directory to make running `parallel` easier
ln -s ../*.ctx .

set -x
parallel --jobs 12 -I% bigsi bloom % %.bloom ::: *.ctx
set +x

+ parallel --jobs 12 -I% bigsi bloom % %.bloom ::: SRR10140498.ctx SRR10140517.ctx SRR10140545.ctx SRR9842706.ctx


In [35]:
ls

+ ls --color=auto
berkeleydb.yaml  SRR9842706.ctx        test-berkeleydb
bigsi.db         SRR9842706.ctx.bloom


: 1

In [34]:
samples=`for i in *.bloom; do echo -n "-s "; basename $i .ctx.bloom; done`
set -x
bigsi build *.bloom ${samples}
set +x

+ bigsi build SRR9842706.ctx.bloom -s SRR9842706
/home/CSCScience.ca/apetkau/miniconda3/envs/bigsi/lib/python3.7/site-packages/bigsi/__main__.py:45: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  config = yaml.load(infile)
INFO:bigsi.cmds.build:Building index: 0/1
DEBUG:bigsi.cmds.build:Loading /home/CSCScience.ca/apetkau/workspace/bigsi-examples/kmers/bigsi/SRR9842706.ctx.bloom/SRR9842706.ctx.bloom 
DEBUG:bigsi.graph.bigsi:Insert sample metadata
DEBUG:bigsi.graph.bigsi:Create signature index
DEBUG:bigsi.graph.index:Transpose bitarrays
DEBUG:bigsi.graph.index:Insert rows
DEBUG:bigsi.storage.base:set bitarrays
Traceback (most recent call last):
  File "/home/CSCScience.ca/apetkau/miniconda3/envs/bigsi/bin/bigsi", line 10, in <module>
    sys.exit(main())
  File "/home/CSCScience.ca/apetkau/miniconda3/envs/bigsi/lib/python3.7/site-packages/bigsi/__main__.py", line 151, in 

In [26]:
bigsi search GTTTCGTTCTTCCGGCGCGGGCGGTCAGCACGTTAACACCACCGACTCCGCTATCCGTATTACCCACTTGCCGACCGGCATCTTGGTGGAATGCCAGGACGAGC

/home/CSCScience.ca/apetkau/miniconda3/envs/bigsi/lib/python3.7/site-packages/bigsi/__main__.py:45: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  config = yaml.load(infile)
{'query': 'GTTTCGTTCTTCCGGCGCGGGCGGTCAGCACGTTAACACCACCGACTCCGCTATCCGTATTACCCACTTGCCGACCGGCATCTTGGTGGAATGCCAGGACGAGC', 'threshold': 1.0, 'results': [{'percent_kmers_found': 100.0, 'num_kmers': 74, 'num_kmers_found': 74, 'sample_name': 'SRR9842706'}], 'citation': 'http://dx.doi.org/10.1038/s41587-018-0010-1'}
